In [1]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/ratings_test.txt')

In [2]:
### GlobalMaxPooling1D() 레이어 하나만

# post로 pad 하고 진행해보기
# 시간오래걸린다
from konlpy.tag import Mecab
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True) 
    test_data = test_data.dropna(how = 'any') 

    x_train = []
    for sentence in train_data['document']:
        temp_x = tokenizer.morphs(sentence) # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
        x_train.append(temp_x)

    x_test = []
    for sentence in test_data['document']:
        temp_x = tokenizer.morphs(sentence) # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
        x_test.append(temp_x)

    words = np.concatenate(x_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(5000-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)} # enumerate 열거하다
#     index_to_word = {index:word for word, index in word_to_index.items()}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

    x_train = list(map(wordlist_to_indexlist, x_train))
    x_test = list(map(wordlist_to_indexlist, x_test))

    return x_train, np.array(list(train_data['label'])), x_test, np.array(list(test_data['label'])), word_to_index

x_train, y_train, x_test, y_test, word_to_index = load_data(train_data, test_data)

print(len(x_train))
print(len(x_test))

146182
49157


In [3]:
total_data_text = list(x_train) + list(x_test)
# 텍스트 데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + round(2.5 * np.std(num_tokens))
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens)*100 / len(num_tokens)))

문장길이 평균 :  15.969376315021577
문장길이 최대 :  116
문장길이 표준편차 :  12.843535456326455
pad_sequences maxlen :  47
전체 문장의 95.2175448835102%가 maxlen 설정값 이내에 포함됩니다. 


In [4]:
# post
x_train = keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value=word_to_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=maxlen)

x_test = keras.preprocessing.sequence.pad_sequences(x_test,
                                                      value=word_to_index["<PAD>"],
                                                     padding='post',
                                                      maxlen=maxlen)

In [5]:
vocab_size = 5000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 8  # 워드 벡터의 차원수 (변경가능한 하이퍼파라미터)

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 8)           40000     
_________________________________________________________________
global_max_pooling1d (Global (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 40,081
Trainable params: 40,081
Non-trainable params: 0
_________________________________________________________________


verbose: Integer. 0, 1, or 2. 
Verbosity mode. 


0 = silent, 
1 = progress bar, 
2 = one line per epoch.

In [6]:
# model 학습 시작 "complie"
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print('---------')
              
epochs=15  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

# fit에 주목
history = model.fit(x_train,
                    y_train,
                    epochs=epochs,
                    batch_size=60,
                    validation_split=0.2,
                    verbose=1)

---------
Epoch 1/15
1950/1950 [==============================] - 3s 2ms/step - loss: 0.4627 - accuracy: 0.7853 - val_loss: 0.3816 - val_accuracy: 0.8304
Epoch 2/15
1950/1950 [==============================] - 3s 2ms/step - loss: 0.3568 - accuracy: 0.8435 - val_loss: 0.3700 - val_accuracy: 0.8390
Epoch 3/15
1950/1950 [==============================] - 3s 2ms/step - loss: 0.3347 - accuracy: 0.8547 - val_loss: 0.3696 - val_accuracy: 0.8379
Epoch 4/15
1950/1950 [==============================] - 3s 2ms/step - loss: 0.3214 - accuracy: 0.8620 - val_loss: 0.3706 - val_accuracy: 0.8392
Epoch 5/15
1950/1950 [==============================] - 3s 2ms/step - loss: 0.3122 - accuracy: 0.8669 - val_loss: 0.3733 - val_accuracy: 0.8394
Epoch 6/15
1950/1950 [==============================] - 3s 2ms/step - loss: 0.3050 - accuracy: 0.8695 - val_loss: 0.3765 - val_accuracy: 0.8379
Epoch 7/15
1950/1950 [==============================] - 3s 2ms/step - loss: 0.2990 - accuracy: 0.8723 - val_loss: 0.3784 - val

In [7]:
# test 는 "evaluate"
results = model.evaluate(x_test,  y_test, verbose=2)

print(results)

1537/1537 - 1s - loss: 0.4068 - accuracy: 0.8313
[0.4067894518375397, 0.8312956690788269]


In [8]:
#